##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Сверточный вариационный автоэнкодер(Convolutional VAE)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/generative/cvae">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/cvae.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/cvae.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/generative/cvae.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

Этот руководство демонстрирует, как обучить вариационный автоэнкодер (VAE) ([1](https://arxiv.org/abs/1312.6114), [2](https://arxiv.org/abs/1401.4082)) на наборе данных MNIST. 
VAE - это вероятностный тип автоэнкодера, модель, которая принимает входные данные большого размера и сжимает их в меньшее представление. В отличие от традиционного автоенкодера, который отображает входные данные в скрытый вектор, VAE отображает входные данные в параметры распределения вероятностей, такие как среднее значение и дисперсия Гаусса. Такой подход создает непрерывное структурированное скрытое пространство, которое полезно для генерации изображений.

![Скрытое пространство CVAE](images/cvae_latent_space.jpg)

## Установка

In [ ]:
!pip -V

In [ ]:
!pip install tensorflow-probability

# to generate gifs
!pip install imageio
!pip install git+https://github.com/tensorflow/docs

In [ ]:
from IPython import display

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import tensorflow_probability as tfp
import time

## Загрузка датасета MNIST
Каждое изображение MNIST изначально представляет собой вектор из 784 целых чисел, каждое из которых находится в диапазоне от 0 до 255 и представляет интенсивность пикселя. Мы нормализуем каждый пиксель изображения и бинаризуем набор данных.

In [ ]:
(train_images, _), (test_images, _) = tf.keras.datasets.mnist.load_data()

In [ ]:
def preprocess_images(images):
  images = images.reshape((images.shape[0], 28, 28, 1)) / 255.
  return np.where(images > .5, 1.0, 0.0).astype('float32')

train_images = preprocess_images(train_images)
test_images = preprocess_images(test_images)

In [ ]:
train_size = 60000
batch_size = 32
test_size = 10000

## Используйте *tf.data* для пакетной обработки и перемешивания данных

In [ ]:
train_dataset = (tf.data.Dataset.from_tensor_slices(train_images)
                 .shuffle(train_size).batch(batch_size))
test_dataset = (tf.data.Dataset.from_tensor_slices(test_images)
                .shuffle(test_size).batch(batch_size))

## Определение нейронной сети енкодера и декодера с помощью *tf.keras.Sequential*

В нашем примере VAE мы используем две небольшие ConvNets для енкодера и декодера. В литературе эти сети также называются моделями вывода/распознавания и генеративными моделями соответственно. Мы используем `tf.keras.Sequential` для упрощения реализации. Пусть $x$ и $z$ обозначают наблюдаемую и скрытую переменные соответственно.

### Нейронная сеть енкодера
Эта сеть берет приблизительное апостериорное распределение $q(z|x)$, которое принимает в качестве ввода наблюдение и выводит набор параметров для определения условного распределения скрытого представления $z$.
В этом примере мы просто моделируем распределение как диагональ гауссовских процессов, и сеть выводит средние и логарифмические отклонения факторизованного гаусса.
Мы выводим логарифмическое отклонение вместо простого отклонения непосредственно для числовой устойчивости.

### Нейронная сеть декодера
Это сеть определяет условное распределение наблюдения $p(x|z)$, которое принимает скрытое представление $z$ в качестве входных данных и выводит параметры для условного распределения наблюдения.
Мы моделируем скрытое априорное распределение $p(z)$ как гауссову единицу.

### Трюк с повторной параметризацией
Чтобы сгенерировать выборку $z$ для декодера во время обучения, мы можем сделать выборку из скрытого распределения, определенного параметрами, выводимыми енкодером, с учетом входного наблюдения $x$.
Однако эта операция выборки создает узкое место, поскольку обратное распространение не может проходить через случайный узел.

Чтобы решить эту проблему, мы используем прием повторной параметризации.
В нашем примере мы аппроксимируем $z$ с использованием параметров декодера и дополнительного параметра $\epsilon$ следующим образом:

$$z = \mu + \sigma \odot \epsilon$$

где $\mu$ и $\sigma$ представляют среднее и стандартное отклонение гауссовского распределения. Их можно получить из выходных данных декодера. $\epsilon$ можно рассматривать как случайный шум, используемый для поддержания стохастичности $z$. Мы генерируем $\epsilon$ из стандартного нормального распределения.

Скрытая переменная $z$ теперь генерируется функцией $\mu$, $\sigma$ и $\epsilon$, что позволяет модели распространять градиенты в декодере обратно через $\mu$ и $\sigma$ , сохраняя стохастичность через $\epsilon$.

### Сетевая архитектура
Для сети енкодера мы используем два сверточных слоя, за которыми следует полносвязный слой. В сети декодера мы отражаем эту архитектуру, используя полносвязный слой, за которым следуют три сверточных слой и слой транспонирования(в некоторых случаях также называемые деконволюционными слоями). Обратите внимание, что при обучении VAE, обычно не используется пакетная нормализация, так как дополнительная случайность из-за использования мини-пакетов может увеличить нестабильность выборки.


In [ ]:
class CVAE(tf.keras.Model):
  """Convolutional variational autoencoder."""

  def __init__(self, latent_dim):
    super(CVAE, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
            tf.keras.layers.Conv2D(
                filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
            # No activation
            tf.keras.layers.Dense(latent_dim + latent_dim),
        ]
    )

    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=32, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            # No activation
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=3, strides=1, padding='same'),
        ]
    )

  @tf.function
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  def encode(self, x):
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

## Определение функции потерь и оптимайзера

VAE обучается, максимизируя нижнюю границу доказательства(ELBO) на логарифме предельного правдоподобия:

$$\log p(x) \ge \text{ELBO} = \mathbb{E}_{q(z|x)}\left[\log \frac{p(x, z)}{q(z|x)}\right].$$

На практике мы оптимизируем оценку этого ожидания методом Монте-Карло для одной выборки:

$$ \ log p (x | z) + \ log p (z) - \ log q (z | x), $$
где $ z $ выбирается из $ q (z | x) $.

**Примечание**: мы также можем аналитически вычислить KL(расстояние Кульбака-Лейблера), но здесь мы включили все три параметра в оценщик Монте-Карло.

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)


def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)


def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z)
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)


@tf.function
def train_step(model, x, optimizer):
  """
    Выполняется на шаге обучение и возвращает величину потерь.
    Эта функция вычисляет потери и градиенты и использует 
    последние для обновления параметров модели.
  """
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

## Обучение

* Мы начинаем с итерации по набору данных
* Во время каждой итерации мы передаем изображение в енкодер, чтобы получить набор среднего и логарифмического приближенного распределения $q(z|x)$
* Затем мы применяем трюк *повторной параметризации* к выборке из $q(z|x)$
* Наконец, мы передаем повторно параметризованные выборки в декодер, чтобы получить вероятности генеративного распределения $p(x|z)$
* **Примечание:** Поскольку мы используем датасет keras, с 60 тыс. примеров данных в обучающем датасете и 10 тыс. в тестовом датасете, наш результирующий ELBO на тестовом датасете немного выше, чем результаты, представленные в литературе, которая использует динамическую бинаризацию MNIST Larochelle.

### Генерация изображений

* После обучения пришло время сгенерировать несколько изображений
* Мы начинаем с получения набора скрытых векторов из гауссовской априорной вероятности $p(z)$
* Затем генератор преобразует скрытую выборку $z$ в вероятности наблюдения, давая распределение $p(x|z)$
* И наконец мы выводим график вероятности распределений Бернулли.


In [ ]:
epochs = 10
# преобразовать размерность скрытого пространства в плоскость для последующей визуализации
latent_dim = 2
num_examples_to_generate = 16

# keeping the random vector constant for generation (prediction) 
# so it will be easier to see the improvement.
random_vector_for_generation = tf.random.normal(
    shape=[num_examples_to_generate, latent_dim])
model = CVAE(latent_dim)

In [ ]:
def generate_and_save_images(model, epoch, test_sample):
  mean, logvar = model.encode(test_sample)
  z = model.reparameterize(mean, logvar)
  predictions = model.sample(z)
  fig = plt.figure(figsize=(4, 4))

  for i in range(predictions.shape[0]):
    plt.subplot(4, 4, i + 1)
    plt.imshow(predictions[i, :, :, 0], cmap='gray')
    plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
# Выберите образец из тестового датасета для генерации выходных изображений
assert batch_size >= num_examples_to_generate
for test_batch in test_dataset.take(1):
  test_sample = test_batch[0:num_examples_to_generate, :, :, :]

In [ ]:
generate_and_save_images(model, 0, test_sample)

for epoch in range(1, epochs + 1):
  start_time = time.time()
  for train_x in train_dataset:
    train_step(model, train_x, optimizer)
  end_time = time.time()

  loss = tf.keras.metrics.Mean()
  for test_x in test_dataset:
    loss(compute_loss(model, test_x))
  elbo = -loss.result()
  display.clear_output(wait=False)
  print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
        .format(epoch, elbo, end_time - start_time))
  generate_and_save_images(model, epoch, test_sample)

### Отображение сгенерированного изображения из последней эпохи обучения

In [ ]:
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
plt.imshow(display_image(epoch))
plt.axis('off')  # Display images

### Отображение анимированног GIF-а всех сохраненных изображений

In [ ]:
anim_file = 'cvae.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)

### Отображение двумерного множества цифр из скрытого пространства

Выполнение приведенного ниже кода покажет непрерывное распределение различных классов цифр, при этом каждая цифра трансформируется в другую в двухмерном скрытом пространстве. Мы используем [TensorFlow Probability](https://www.tensorflow.org/probability), чтобы сгенерировать стандартное нормальное распределение для скрытого пространства.

In [ ]:
def plot_latent_images(model, n, digit_size=28):
  """Plots n x n digit images decoded from the latent space."""

  norm = tfp.distributions.Normal(0, 1)
  grid_x = norm.quantile(np.linspace(0.05, 0.95, n))
  grid_y = norm.quantile(np.linspace(0.05, 0.95, n))
  image_width = digit_size*n
  image_height = image_width
  image = np.zeros((image_height, image_width))

  for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
      z = np.array([[xi, yi]])
      x_decoded = model.sample(z)
      digit = tf.reshape(x_decoded[0], (digit_size, digit_size))
      image[i * digit_size: (i + 1) * digit_size,
            j * digit_size: (j + 1) * digit_size] = digit.numpy()

  plt.figure(figsize=(10, 10))
  plt.imshow(image, cmap='Greys_r')
  plt.axis('Off')
  plt.show()

In [ ]:
plot_latent_images(model, 20)

## Следующие шаги

В этом руководстве показано, как реализовать сверточный вариационный автоэнкодер с помощью TensorFlow.

В качестве следующего шага вы можете попытаться улучшить вывод модели, увеличив размер сети.
Например, вы можете попробовать установить параметры `filter` для каждого из слоев` Conv2D` и `Conv2DTranspose` на 512.
Обратите внимание, что для генерации окончательного 2D-графика скрытого изображения вам нужно оставить для параметра `latent_dim` значение 2. Кроме того, время обучения будет увеличиваться с увеличением размера сети.

Вы также можете попробовать реализовать VAE, используя другой набор данных, например CIFAR-10.

VAE могут быть реализованы в нескольких различных стилях и различной сложности. Вы можете найти дополнительные реализации в следующих источниках:
- [Вариационный автоэнкодер(keras.io)](https://keras.io/examples/generative/vae/)
- [Пример VAE из руководства «Написание пользовательских слоев и моделей»(tensorflow.org)](https://www.tensorflow.org/guide/keras/custom_layers_and_models#putting_it_all_to General_an_end-to-end_example)
- [Вероятностные слои TFP: вариационный автоэнкодер](https://www.tensorflow.org/probability/examples/Probabilistic_Layers_VAE)

Если вы хотите узнать больше о VAE, обратитесь к [Введение в вариационные автоэнкодеры](https://arxiv.org/abs/1906.02691).